# California Spotted Owl Data
### Column entries
* SNAME - Strix occidentalis (occidentalis/caurina)
* CNAME - (California/Northern) Spotted Owl
* OBSID - observation ID
* MASTEROWL - MasterOwl Number is the unique name given to each site where the observation occured
    * If an observation is near a recorded detection and there are no topographical barriers, then the observation will get the same MasterOwl Number as the nearby detection.
    * If an observation is far from a recorded detection or there are topographical barriers then the observation will be given a MasterOwl value of POS (for a positive observation) or NEG (for a negative observation).
* TYPEOBS - TypeObs field: positive observation (value: POS), negative observation (value: NEG), and activity center (value: AC).
    * A positive observation, or detection, means that bird(s) were documented at a given location.
    * A negative observation means that a survey was conducted but no birds were found.
    * The AC value is assigned to the highest-ranking positive observation
* DATEOBS - Date Observed
* NUMADOBS - TBD
* AGESEX - TBD
* PAIR - Pair
* NEST - Nest Observation
* NUMYOUNG - Number of young species
* SUBSPECIES - Subspecie category (C or N)
* LONDD_N83 - longitude
* LATDD_N83 - latitude
* DOCID - TBD
* MTRS - TBD
* HIGHESTUSE - TBD
* SYMBOLOGY - TBD
* COORDS - Coordinate tuple of the observation

### Excluded Column Entries
* OBSERVER
* TIMEOBS
* COORDSRC
* COMMENTS

In [1]:
import fiona, os
import pandas as pd
os.chdir("..")

In [4]:
file_name = "test_data/spowdb"

In [10]:
# Read the entire shape file
with fiona.open(file_name) as shape:
    #schema = shape.schema
    df = []
    for obs in shape:
        # main properties
        if not 'properties' in obs.keys(): continue
        df.append(dict(obs['properties']))
        # coordinates
        if not 'geometry' in obs.keys(): continue
        if not 'coordinates' in obs['geometry']: continue
        df[-1]['COORDS'] = obs['geometry']['coordinates']
    pdf = pd.DataFrame(df)

In [7]:
pdf.sample(10)

,SNAME,CNAME,OBSID,MASTEROWL,TYPEOBS,OBSERVER,DATEOBS,TIMEOBS,NUMADOBS,AGESEX,...,SUBSPECIES,LONDD_N83,LATDD_N83,COORDSRC,DOCID,COMMENTS,MTRS,HIGHESTUSE,SYMBOLOGY,COORDS
63197,Strix occidentalis occidentalis,California Spotted Owl,36651,MAD0072,POS,SIE NF,1991-07-07,None,1,UU,...,C,-119.375349,37.432684,Quarter-section centroid,"1409, 1579",None,M 06S 24E 05,POS,POS,"(55195.756999999285, -64703.73589999974)"
178198,Strix occidentalis caurina,Northern Spotted Owl,104584,TRI0100,NEG,KALSO ET AL.,1989-07-27,None,0,None,...,N,-123.240637,40.281967,Half-section centroid,"6291, 6568",None,M 28N 12W 17,NEG,NEG,"(-275475.54099999927, 256486.67750000022)"
106754,Strix occidentalis caurina,Northern Spotted Owl,163367,MEN0679,NEG,BRINKERHOFF,2016-06-04,0715-1005,0,None,...,N,-123.193266,39.908558,Contributor,7311,no response,M 24N 12W 28,NEG,NEG,"(-272846.9949999992, 214895.49290000089)"
124128,Strix occidentalis caurina,Northern Spotted Owl,141378,NEG,NEG,WOOSTER,2004-04-12,2400,0,None,...,N,-123.490245,38.975856,Section centroid,7159,no response,M 13N 15W 23,NEG,NEG,"(-302025.8959999997, 112215.88759999909)"
92273,Strix occidentalis caurina,Northern Spotted Owl,45002,MEN0374,NEG,TREADWAY,2000-04-06,2307-2317,0,None,...,N,-123.562560,39.440280,Contributor,1063,no response,M 18N 15W 05,NEG,NEG,"(-306341.6970000006, 164026.23039999977)"
154325,Strix occidentalis occidentalis,California Spotted Owl,88355,SIE0040,POS,BAKOS-BENEFIELD,2001-05-25,0016,1,UF,...,C,-120.474224,39.623235,Contributor,2420,"CP7,FA CHATTERING,MA NEARBY.",M 20N 13E 02,PAIR,PAIR,"(-40685.88479999825, 178701.4252000004)"
56730,Strix occidentalis caurina,Northern Spotted Owl,120889,HUM1049,NEG,BRICKER,2012-05-09,0102-0112,0,None,...,N,-123.700056,40.224375,Contributor,6838,frogs; no response,H 03S 04E 11,NEG,NEG,"(-314760.16410000063, 251524.46759999916)"
182000,Strix occidentalis caurina,Northern Spotted Owl,181008,TRI0282,POS,via USFS NRIS - STNF,2014-08-20,1624,1,AF,...,N,-123.070090,40.681123,Contributor,0007,Banded: RED/WHT dia on L. This F nested and re...,M 33N 11W 26,POS,POS,"(-259548.06079999916, 300305.51249999925)"
81550,Strix occidentalis caurina,Northern Spotted Owl,49986,MEN0192,POS,MDNF,1988-08-15,None,1,UU,...,N,-122.952135,39.837764,Quarter-section centroid,7557?,None,M 23N 10W 22,POS,POS,"(-252495.46880000085, 206366.58720000088)"
101153,Strix occidentalis caurina,Northern Spotted Owl,62777,MEN0538,NEG,SF,2003-06-12,1956,0,None,...,N,-123.674490,39.214770,Quarter-section centroid,"6112, 6139",None,M 16N 16W 29,NEG,NEG,"(-316932.7639000006, 139351.48139999993)"


3 cols: lat long, numobs
group by lat long and sum to combine owl groups.

round latlong to 2 decimal places (increase range of observation)

In [21]:
pdf.to_csv('test_data/spowdb/Spotted_Owl_Observations.csv')

## Produce Observation Ratios
### Note: Currently the observation values are used (not observatio ratios), since no "total checklist" per square data is available 

In [67]:
pdf = pd.read_csv("test_data/spowdb/Spotted_Owl_Observations.csv", index_col=0)
#pdf['NUMADOBS'] = pdf['NUMADOBS'].fillna(0).astype('int')

/var/folders/k_/wkz354ws3rj9g1dwmlg8ftm40000gn/T/ipykernel_32062/2269438793.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,10,11,12,14,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  pdf = pd.read_csv("test_data/spowdb/Spotted_Owl_Observations.csv", index_col=0)


In [68]:
# Round LAT LOND entries to 2 decimal points
pdf[['LONDD_N83', 'LATDD_N83']] = pdf[['LONDD_N83', 'LATDD_N83']].round(2)
# Get the Total Number of Observations (POS, NEG, AC)
total_obs = len(pdf)

In [70]:
# Get Observation Ratios (Positive & AC)
pos_pdf = pdf.loc[(pdf['TYPEOBS']=='POS') | (pdf['TYPEOBS']=='AC')]
# Group and sum nearby observations
pos_pdf = pos_pdf.groupby(['LONDD_N83', 'LATDD_N83'])['NUMADOBS'].sum()
# Output Obs Ratios to txt
output = "["
for i, v in pos_pdf.iteritems():
    output += f"[\"{i[1]};{i[0]}\", {v}], "
output = output[:-2]
output += "]"
file = open("test_data/spowdb/pos_observations.txt", "w")
file.write(output)
file.close()